<a href="https://colab.research.google.com/github/rauladamson/Kodutoo_GenAI/blob/dev/genai_praktika_kodut%C3%B6%C3%B6_2025_RAUL_ADAMSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kodutöö GenAI praktika kandidaadile
Aprill 2025

**Kandideerija nimi:** Raul Adamson

Juhised:
* Palun loo sellest failist koopia ning kirjuta faili nimesse oma nimi.
* Kodutöös on kaks ülesannet. Esimesel ülesandel on 9 alamülesannet. Teine ülesanne on valikuline ja sellel pole alamülesandeid.
* Lahenda ülesanded ühes Jupyter Notebooki failis. Võid teha juurde code cell'e, installida teeke juurde jne.
* Lahendamisel kasuta Python programmeerimiskeelt.
* Kui kasutad koodijuppe muudest allikatest (nt blogi, dokumentatsioon, StackOverflow), siis viita nendele.
* AI assistendi/chatboti abi koodi kirjutamisel on lubatud, kuid kommentaarid ja põhjendused tuleb kirjutada ise. Oluline on, et saaksid lahendusest ka ise aru.
* Google Gemini API-t ja teisi API-sid peaks kasutama ainult tasuta mahus. Eelistatud on vabavaralised ja tasuta tööriistad.
* Saada lahendatud kodutöö Google Colab faili link või Githubi üles laetud faili link e-mailile kea.kohv@telia.ee. Githubi puhul veendu, et Kea Kohvile (keakohv) on antud repository'le lugemisõigus.
* Veendu, et notebook on läbi jooksutatud ning cell output näha. Me ei jooksuta lahendusi ise läbi.
* Kodutöö lõpus on tagasiside lahter, palun täida ka see.
* Lahendus saada hiljemalt pühapäeval, 27.04.2025 kl 23.59.

# 1. ülesanne

Esimeses ülesandes laeme alla failid ning muudame need tavaliseks tekstiks; kasutame vektorotsingut ning hübriidotsingut, et leida fraasidele kõige relevantsemaid tekstijuppe; genereerime mudeliga vastuseid nii ilma kontekstita kui ka kontekstiga ehk koos relevantse infoga.

## 1.1 alamülesanne

Lae alla need 2 PDF faili:

https://www.telia.ee/images/documents/TELIA_raport_EST_2024.pdf
https://www.telia.ee/images/documents/hinnakirjad/est/telia_eesti_ariteenused_yldosa.pdf

Samuti lae alla selle veebilehe sisu HTML formaadis:
https://digitark.telia.ee/era/rohde-schwarz-eestis-on-mobiilse-interneti-kiirus-huppeliselt-kasvanud/


In [8]:
import requests
import os
from urllib.parse import urlparse

def download_file(url, save_dir='downloads'):
    try:
        # Create directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Get the filename from URL
        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)
        if not filename:
            filename = "downloaded_file.html"  # Default name if no filename found

        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes

        # Save the file
        file_path = os.path.join(save_dir, filename)
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # filter out keep-alive new chunks
                    f.write(chunk)

        print(f"Successfully downloaded: {file_path}")
        return file_path

    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

# List of URLs to download
urls = [
    "https://www.telia.ee/images/documents/TELIA_raport_EST_2024.pdf",
    "https://www.telia.ee/images/documents/hinnakirjad/est/telia_eesti_ariteenused_yldosa.pdf",
    "https://digitark.telia.ee/era/rohde-schwarz-eestis-on-mobiilse-interneti-kiirus-huppeliselt-kasvanud/"
]

# Download all files
for url in urls:
    download_file(url)
    print("All files downloaded.")

KeyboardInterrupt: 

## 1.2 alamülesanne

Konverteeri allalaetud kaks PDF faili ja üks HTML fail MarkDown formaati ehk tavaliseks tekstiks. Pilte ei ole vaja konverteerida. Prindi välja igast MarkDown failist esimesed 2000 tähemärki.

In [7]:
!pip install extractous markdownify
import os
from pathlib import Path
from docling.document_converter import DocumentConverter  # Correct import
from extractous import Extractor
from markdownify import markdownify as md

# Setup directories
input_dir = 'downloads'
output_dir = 'markdown_output'
os.makedirs(output_dir, exist_ok=True)

# Initialize extractor
extractor = Extractor()

def clean_filename(filename):
    """Clean filename for safe saving"""
    return "".join(c if c.isalnum() or c in ('-', '_') else '_' for c in filename)

def process_file(filepath):
    """Process a single file with both methods"""
    filename = Path(filepath).name
    print(f"\nProcessing: {filename}")

    results = {}

    # Method 1: Docling conversion
    try:
        # Correct docling usage with DocumentConverter
        converter = DocumentConverter()
        result = converter.convert(filepath)
        docling_md = result.document.export_to_markdown()
        results['docling'] = docling_md[:2000]  # Store first 2000 chars

        # Save full output
        output_path = os.path.join(output_dir, f"docling_{clean_filename(filename)}.md")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(docling_md)
    except Exception as e:
        print(f"Docling failed: {str(e)}")
        results['docling'] = None

    # Method 2: Extractous + Markdownify
    try:
        if filepath.endswith('.pdf'):
            text, _ = extractor.extract_file_to_string(filepath)
            extracted_md = md(text) if text else ""
        elif filepath.endswith(('.html', '.htm')):
            with open(filepath, 'r', encoding='utf-8') as f:
                html = f.read()
            extracted_md = md(html)
        else:
            print(f"Skipping unsupported file type: {filename}")
            return None

        results['extractous'] = extracted_md[:2000]  # Store first 2000 chars
        # Save full output
        output_path = os.path.join(output_dir, f"extractous_{clean_filename(filename)}.md")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(extracted_md)
    except Exception as e:
        print(f"Extractous+Markdownify failed: {str(e)}")
        results['extractous'] = None

    return results

# Main processing loop
for filename in os.listdir(input_dir):
    filepath = os.path.join(input_dir, filename)
    if os.path.isfile(filepath):
        results = process_file(filepath)

        if results:
            print("\nComparison for:", filename)
            print("="*50)
            print("Docling output (first 2000 chars):")
            print(results.get('docling', 'No output'))
            print("\nExtractous+Markdownify output (first 2000 chars):")
            print(results.get('extractous', 'No output'))
            print("="*50 + "\n")

print(f"\nAll files processed. Outputs saved to '{output_dir}' directory.")


Processing: downloaded_file.html

Comparison for: downloaded_file.html
Docling output (first 2000 chars):
# Rohde &amp; Schwarz: Eestis on mobiilse interneti kiirus hüppeliselt kasvanud

4 min lugemine

<!-- image -->

09.12.2024 08:01

Rahvusvaheline seirefirma Rohde &amp; Schwarz viis sügisel Eestis läbi ulatuslikud mõõtmised, mille käigus analüüsiti Telia, Elisa ja Tele2 mobiilivõrkude suutlikkust. Selgub, et tänu 5G võrkude arengule on mobiilse interneti keskmine kiirus viimase aastaga Eestis oluliselt kasvanud.

Saamaks kogu riigi kohta võimalikult täpset ja ülevaatlikku pilti, sõitis Rohde &amp; Schwarz mobiilivõrkude testimise nimel terve Eesti risti-põiki läbi. Selle käigus läbiti kokku ca 6500 kilomeetrit ning teostati mõõtmisi nii olulisematel maanteedel kui ka kõigis suuremates linnades eesotsas Tallinna, Tartu, Pärnu, Narva ja Kohtla-Järvega.

Lisaks sellele mõõdeti kõigi kolme operaatori võrkude jõudlust terves reas väiksemates linnades ja asukohtades, sh Viljandi, Maardu


Comparison for: TELIA_raport_EST_2024.pdf
Docling output (first 2000 chars):
<!-- image -->

Telia ühiskondliku mõju raport 2024

Telia

Panustame ühiskonna arengusse, et Eesti oleks parem paik nii elamiseks kui töötamiseks.

Keskkonnategurid

Sotsiaalne mõju

Juhtimisalased tegurid

2

<!-- image -->

## Sisukord

## Keskkond

## Heitme- ja jäätmevabadus

Emissioonid, energia ja liikuvus

Energia

Transport

Jäätmed ja ringmajandus

Jäätmete osakaal, ümbertöötamine ja taaskasutamine

Digikelts

Keskkonnateemad sise- ja väliskommunikatsioonis

Telia töötajate keskkonnateadlikkus

## Sotsiaalne mõju

## Digitaalne kaasatus ning privaatsus ja küberturvalisus

Koostöö riigi ja võrgustikega, teadus- ja arendustegevus

Juhendid, koolitused ja hariduslikud programmid

Telia töötajate privaatsuse ja küberturvalisuse alase teadlikkuse arendamine

Hariduslikud programmid

Ühiskonnauuringud

Sideteenuste kättesaadavus

## Telia tööandjana

Võrdsed võimalused ja igaühega arvestamine

Äriklientid

<font color='purple'>Põhjendus</font> - Miks valisid just sellise MarkDown formaati konverteerimise meetodi/meetodid? Mida nende andmete kohta arvad, kas neid on lihtne või raske teksti konverteerida?
### Meetod
 Arendusprotsessi vältel kaalusin ning katsetasin mitut erinevat meetodit ning teeki:

1.   **[markitdown](https://github.com/microsoft/markitdown)**
  *   Arendatud Microsofti poolt
  *   Paindlik litsents (MIT)
  *   Aeglane
  *   Ebatäpne

2.   **[marker](https://github.com/VikParuchuri/marker)**
  *   Kiire
  *   Lokaalne
  *   [Piirav litsents suurettevõtete jaoks](https://github.com/VikParuchuri/marker?tab=readme-ov-file#commercial-usage) (cc-by-nc-sa-4.0)

3.  **[LlamaParse](https://www.llamaindex.ai/llamaparse)**
  *   Paindlik litsents
  *   Kasutab LlamaParse API'd, mis piirab kiirust ning kasutusi

Lõpuks jäin kahe variandi vahele:

4. **[Extractous](https://https://github.com/yobix-ai/extractous) + [Markdownify](https://pypi.org/project/markdownify/)**
  *  25x kiirem kui unstructured.io, mida algselt plaanisin katsetada
  *  Lokaalne
  *  Pole piisavalt täpne keeruliselt struktureeritud PDF failide töötlemisel
  *  Paindlikud litsentsid (Apache license 2.0 + MIT)

5. **[Docling](https://github.com/docling-project/docling)**
  *  Laeb mudeleid HuggingFace'ist ning jooksutab neid lokaalselt
  *  Palju täpsem kui Extractous
  *  Aeglasem kui Extractous, aga kiirem kui LlamaParse
  *  Toetab teksti tükeldamist, mis vähendab kasutatud teeke
  *  Arendatud IBM'i poolt
  *  Paindlik litsents (MIT)

  Lõpptulemuses otsustasin kasutada Docling'i, sest selles olukorras täpsus on tähtsam kui kiirus.

  ### Raskus
  #### PDF
  PDF failitüübil puudub täpne struktuur, mille tõttu konverteerimisel struktureeritud formaati võivad tekkida vead sõltuvalt PDF faili kujundusest ja keerukusest. Näiteks kui ühel lehel asuvad kõrvuti tekstiread, mida loetakse ülevalt alla. Isegi kõige täpsemate meetoditega võivad tekkida ebatäpsused.
  #### HTML
  HTML on struktureeritud, mille tõttu teisendamine on palju kiirem võrreldes  ning täpsem võrreldes PDF'iga

## 1.3 alamülesanne

Tükelda tekstid juppideks (chunking). Vali ise jupi suurus ja jupitamise strateegia. Prindi igast allikast (algselt 2 PDF-i ja 1 HTML) vähemalt 2 loodud tekstijuppi.

In [ ]:
# to-do

<font color='purple'>Põhjendus</font> - Palun kirjelda, miks valisid sellise jupitamise strateegia:

[KIRJALDA SIIN]

## 1.4 alamülesanne

Vektoriseeri jupid vabalt kättesaadava enda valitud teksti vektoriseerimise mudeliga (embedding model), nt HuggingFace'ist. Eelista väiksemat mudelit. Prindi välja kaks loodud vektorit.

In [ ]:
# to-do

<font color='purple'>Põhjendus</font> - Palun kirjelda, miks valisid sellise vektoriseerimise mudeli.

[KIRJELDA SIIN]

## 1.5 alamülesanne

Leia vektorite sarnasuselt/kauguselt lähimad tekstijupid nendele fraasidele:


    "Mis on allalaadimiskiirus maanteedel?",
    "Mis on vana uueks?",
    "Kes on Andre Visse?"


Iga fraasi puhul prindi välja top 3 sarnaseimat tekstijuppi.



In [ ]:
queries = [
    "Milline on allalaadimiskiirus maanteedel?",
    "Mis on vana uueks?",
    "Kes on Andre Visse?"
]

# to-do

## 1.6 alamülesanne

Kombineeri nüüd vektorite sarnasuse/kauguse otsing terminipõhise (keyword-based) otsinguga ehk tee hübriidotsing. Prindi välja iga fraasi kohta top 3 sarnaseimat tekstijuppi. Kasuta samu fraase nagu eelmises alamülesandes.

Vihje: mõtle ka stoppsõnade peale

In [ ]:
# to-do

<font color='purple'>Põhjendus</font> - Kirjelda, kuidas oled tulemusega rahul. Kas hübriidotsing oli ainult vektorite võrdlemise meetodist parem või mitte? Miks? Kuidas saaks tulemust veelgi parandada?

[KIRJELDA SIIN]

## 1.7. alamülesanne

Vali HuggingFace'ist üks vabalt kättesaadav väike multilingual text-to-text mudel. Genereeri mudeliga vastus nendele sisendküsimustele:

    "Mis on allalaadimiskiirus maanteedel?",
    "Mis on vana uueks?",
    "Kes on Andre Visse?"

Prindi vastused välja.

Eelista väikest mudelit, mida saaks kasutada Colab-iga. Kui vaja, kasuta Colab-i tasuta GPU-d (vaheta käitusaja tüüpi). Alternatiivis on ka [Kaggle](https://www.kaggle.com/)'is nädalas 30 tundi GPU-aega tasuta.

Mudeli mälu vajaduse vähendamiseks võid kasutada kvantiseerimist.

Genereeri vastused kasutades ainult mudelit, ära kasuta mõnda rakendust, API-t ega lisatööriistu.

In [ ]:
# to-do

<font color='purple'>Põhjendus</font> - Kas mudeli vastused tunduvad head? Põhjenda. Kui ei ole head, siis miks? Mida saaks teha, et vastused oleksid paremad? Kui võrdlesid mitut mudelid, võid siin oma avastusi kirjeldada.

[KIRJELDA SIIN]

## 1.8 alamülesanne

Küsi samad küsimused API kaudu mõnelt suuremalt mudelilt. Kasuta Google Gemini API-t, mille kaudu saab iga päev teatud mahu tasuta päringuid teha. Näiteks Gemini 2.0 Flash päringuid saab teha tasuta päevas 1500. Prindi küsimused ja API-lt saadud vastused välja.

API-i key saab genereerida siin: https://aistudio.google.com/apikey

Mudeli rate limit info siin: https://ai.google.dev/gemini-api/docs/rate-limits


In [ ]:
from google.colab import userdata
gemini_api_key = userdata.get('gemini_api_key') # palun vaata, et sa oma API key'd meile ei saada

In [ ]:
# to-so

## 1.9 alamülesanne

Muuda nüüd loogikat nii, et enne suurele keelemudelile (LLM-ile) päringu tegemist kasutad hübriidotsingut, et leida küsimusele hübriidotsinguga top 3 sarnaseimat tekstijuppi. Lisa LLM-i päringusse kontekstina need top 3 relevantset tekstijuppi. Prindi küsimused ja LLM-i vastused. Kasuta jätkuvalt Gemini API-t.

In [ ]:
# to-do

<font color='purple'>Põhjendus</font> - Kui andsid LLM-ile sisendina kaasa ka konteksti, mille põhjal vastata, siis kas vastus oli parem? Miks? Kas kõik vastused on nüüd õiged või võib olla mõnel ka aegunud info? Kuidas saaks aegunud info probleemi parandada?

[KIRJELDA SIIN]

# 2. ülesanne - tööriistu kasutav keelemudel, valikuline ülesanne

See on valikuline ülesanne. Kirjuta kood, mis võimaldab keelemudelil vastamisel kasutada mõnda tööriista (nn tool calling / function calling). Näiteks keelemudel teeb ise API päringu(id), et saada vastamiseks lisainfot. Tööriista(d), mida keelemudel ülesande lahendamiseks kasutab, võid valida ise. Samuti võid valida ise ülesande või küsimuse, mida keelemudel lahendama peab. Võid kasutada ka mitut keelemudeli päringut järjest või luua nn multi-agent süsteemi.

In [ ]:
# to-do

<font color='purple'>Põhjendus (kui ülesannet tegid) </font> - Mida sinu tööriista(sid) kasutava keelemudeliga teha saab?

[KIRJELDA SIIN]

# Tagasiside

<font color='purple'>Tagasiside</font> - Kirjuta vabas vormis, mis oli sinu jaoks lihtne ning mis keerulisem. Lisaks, mis osa oli kõige põnevam ning mida õppisid.

[KIRJELDA SIIN]